# Two Sample Topo Test For Cyclostationary Signals

This notbook shows how to process cyclostationary signals for use with the TopoTest code base.

In [2]:
import sys
import pandas as pd
import numpy as np
sys.path.append('topotests/')
from topotests.topotests import TopoTestTwosample

# import some random variables (RV) generators
from scipy.stats import levy_stable
# import univariate Kolmogorov-Smirnov test
from scipy.stats import ks_2samp

# set random number generator seed for reproducibility
np.random.seed(seed=12345)

A periodic autoregressive model of order 1 (PAR(1)) is defined by $X_t = \phi_{t}X_{t-1} + \varepsilon_t$, where $\varepsilon_t\sim \mathcal{N}(0,1)$.
The periodicity is built into the coefficients via $\phi_t = \phi_{t+T}$, with $T$ being the period.
We are also interested in processes $Y_t = X_t + \sigma_Z Z_t$, where $X_t$ is PAR(1), $Z_t$ is stationary $\alpha$-stable noise and $\sigma_Z>0$.

In [7]:
def PAR1_gauss(phi,sigmaE,N,seed):
    #phi - vector of coefficients phi_1(1),...,phi_1(T)
    #sigmaE - st. deviation of innovations (they have N(0,sigmaE) distribution)
    #N - sample length
    rng = np.random.default_rng(seed=seed)
    T = len(phi)
    periods_before = 50
    E = rng.normal(0,sigmaE,N+T*periods_before)
    X = np.empty((N+T*periods_before,))
    X[0] = E[0]
    for j in range(1,N+T*periods_before):
        ind = j%T
        X[j] = phi[ind]*X[j-1] + E[j]
    X = X[periods_before*T:]
    return X

def create_signal(phi, seed, N=1000, sigmaE=1, alpha=2, sigmaZ=0.8):
    X = PAR1_gauss(phi,sigmaE,N,seed=seed) #"pure" PAR1 sample (no additive noise yet)
    Z = sigmaZ*levy_stable.rvs(alpha,0,loc=0,scale=1,size=N, random_state=seed)
    Y = X + Z # final sample: PAR1 with additive noise

    return Y

Let us sample different trajectories.

In [19]:
W = create_signal([-0.4,0.6,0.8,-0.7],seed=0)
X = create_signal([-0.4,0.6,0.8,-0.7],seed=1)
Y = create_signal([-0.4,0.6,0.8,-0.7], sigmaZ=0, seed=2)
Z = create_signal([0.5,0.7,-0.9,0.6], sigmaZ=0, seed=3)

Now Topotest works in the spatial domain, into which the time series need to be transformed. Regarding blocks of length $T$ successive observations as a point in $\mathbb{R}^T$, we construct a stationary point process:
$$
    (X_1, X_2, ...) \mapsto \{(X_1,...,X_T), (X_{T+1}, ... ,X_{2T}), ...\}.
$$

In [20]:
embW = np.array([W[i:i+4] for i in range(0,len(W),4)])
embX = np.array([X[i:i+4] for i in range(0,len(X),4)])
embY = np.array([Y[i:i+4] for i in range(0,len(Y),4)])
embZ = np.array([Z[i:i+4] for i in range(0,len(Z),4)])

Note however, that this point process is very much not spatially independent. Let us find out whether the Topotest still works.

In [21]:
print(TopoTestTwosample(X1=embW, X2=embX))

TopoTestResult(statistic=0.15200000000000014, pvalue=0.732)


In [18]:
print(TopoTestTwosample(X1=embX, X2=embY))
print(TopoTestTwosample(X1=embX, X2=embZ))
print(TopoTestTwosample(X1=embY, X2=embZ))
      

TopoTestResult(statistic=0.7080000000000006, pvalue=0.0)
TopoTestResult(statistic=0.7200000000000006, pvalue=0.0)
TopoTestResult(statistic=0.1160000000000001, pvalue=0.958)


The results seem to indicate that TopoTest is able to distinguish between the pure PAR(1) model and the one with added noise independently from the choice of coefficients $\phi$.